In [ ]:
from statistics import median
import pandas as pd
import json
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Leer los datos del archivo JSON
data_raw = pd.read_json('./Ventas_registro.json')

In [ ]:
# Desanidamos los productos para usar los valores anidados (nombre - cantidad - total)
data_flat = pd.json_normalize(data_raw.to_dict(orient='records'), record_path='productos', meta=['tipo_venta', 'fecha_venta', 'pago_total', 'cliente'])

### TEMA 8

#### DEFINICIÓN DE VARIABLES CANTIDAD, TOTAL, PAGO_TOTAL

In [ ]:
sns.boxplot(data=data_flat, x='cantidad', legend='auto')
plt.title('Diagrama de Caja - Cantidad')
plt.xlabel('Total de cantidad del Producto')
plt.show()

In [ ]:
sns.boxplot(data=data_flat, x='total', legend='auto')
plt.title('Diagrama de Caja - Total')
plt.xlabel('Valor total del producto')
plt.show()

In [ ]:
sns.boxplot(data=data_flat, x='pago_total', legend='auto')
plt.title('Diagrama de Caja - Pago Total')
plt.xlabel('Pago total de las ventas')
plt.show()

#### Z-SCORE

In [183]:
from scipy.stats import zscore

# Calcular el Z-score para `cantidad` y `total`
data_flat['z_cantidad'] = zscore(data_flat['cantidad'])
data_flat['z_total'] = zscore(data_flat['total'])

# Filtrar los outliers (valores cuyo Z-score > 3 o <-3)
data_flat_clean = data_flat[(data_flat['z_cantidad'].abs() <= 3) & (data_flat['z_total'].abs() <= 3)]

# Eliminar las columnas z_cantidad y z_total para mantener la estructura original
data_flat_clean = data_flat_clean.drop(['z_cantidad', 'z_total'], axis=1)

# Ver los primeros registros de la tabla limpia
print(data_flat_clean.shape)

(29072, 7)


In [185]:
# Número de filas en el dataframe original
total_original = data_flat.shape[0]

# Número de filas en el dataframe limpio (sin outliers)
total_clean = data_flat_clean.shape[0]

# Cantidad de valores outliers eliminados
outliers_count = total_original - total_clean

print(f"Cantidad de outliers encontrados: {outliers_count}")


Cantidad de outliers encontrados: 276


In [184]:
from scipy.stats import zscore

# Calcular el Z-score para la columna `pago_total`
data_raw['z_pago_total'] = zscore(data_raw['pago_total'])

# Filtrar los outliers (valores cuyo Z-score > 3 o <-3)
data_raw_clean = data_raw[data_raw['z_pago_total'].abs() <= 3]

# Eliminar la columna z_pago_total para mantener la estructura original
data_raw_clean = data_raw_clean.drop(['z_pago_total'], axis=1)

# Ver los primeros registros de la tabla limpia
print(data_raw_clean.shape)

(9770, 5)


In [186]:
# Número de filas en el dataframe original
total_original = data_raw.shape[0]

# Número de filas en el dataframe limpio (sin outliers)
total_clean = data_raw_clean.shape[0]

# Cantidad de valores outliers eliminados
outliers_count = total_original - total_clean

print(f"Cantidad de outliers encontrados: {outliers_count}")

Cantidad de outliers encontrados: 45


In [ ]:
# Boxplot de `pago_total` antes y después de limpiar outliers
plt.figure(figsize=(12, 6))

# Boxplot para `pago_total` en data_raw (antes de limpiar)
sns.boxplot(x='status', y='pago_total', data=pd.DataFrame({
    'pago_total': data_raw['pago_total'],
    'status': ['Antes'] * len(data_raw)
}), color='lightblue', width=0.4)

# Boxplot para `pago_total` en data_raw_clean (después de limpiar)
sns.boxplot(x='status', y='pago_total', data=pd.DataFrame({
    'pago_total': data_raw_clean['pago_total'],
    'status': ['Después'] * len(data_raw_clean)
}), color='lightgreen', width=0.4)

plt.title('Comparativa de Boxplot de pago_total: Antes vs Después de limpiar outliers')
plt.ylabel('Pago Total')
plt.show()

In [ ]:
# Boxplot para `cantidad` en data_flat (antes y después de limpiar outliers)
plt.figure(figsize=(12, 6))

# Boxplot para `cantidad` en data_flat (antes de limpiar)
sns.boxplot(x='status', y='cantidad', data=pd.DataFrame({
    'cantidad': data_flat['cantidad'],
    'status': ['Antes'] * len(data_flat)
}), color='lightblue', width=0.4)

# Boxplot para `cantidad` en data_flat_clean (después de limpiar)
sns.boxplot(x='status', y='cantidad', data=pd.DataFrame({
    'cantidad': data_flat_clean['cantidad'],
    'status': ['Después'] * len(data_flat_clean)
}), color='lightgreen', width=0.4)

plt.title('Comparativa de Boxplot de cantidad: Antes vs Después de limpiar outliers')
plt.ylabel('Cantidad')
plt.show()

In [ ]:
# Boxplot para `total` en data_flat (antes y después de limpiar outliers)
plt.figure(figsize=(12, 6))

# Boxplot para `total` en data_flat (antes de limpiar)
sns.boxplot(x='status', y='total', data=pd.DataFrame({
    'total': data_flat['total'],
    'status': ['Antes'] * len(data_flat)
}), color='lightblue', width=0.4)

# Boxplot para `total` en data_flat_clean (después de limpiar)
sns.boxplot(x='status', y='total', data=pd.DataFrame({
    'total': data_flat_clean['total'],
    'status': ['Después'] * len(data_flat_clean)
}), color='lightgreen', width=0.4)

plt.title('Comparativa de Boxplot de total: Antes vs Después de limpiar outliers')
plt.ylabel('Total')
plt.show()

### TEMA 9

#### PREPARACIÓN DE DATOS - CONSTRUCCIÓN

##### GENERACIÓN DE DATOS

1. ATRIBUTOS DERIVADOS

In [ ]:
# Calcular el precio unitario de cada producto
data_flat_clean['precio_unitario'] = data_flat_clean['total'] / data_flat_clean['cantidad']
data_flat_clean[['nombre', 'cantidad', 'total', 'precio_unitario']].head()

In [ ]:
# Total de productos vendidos en una venta
data_raw_clean['total_productos'] = data_flat_clean.groupby(['fecha_venta', 'cliente'])['cantidad'].transform('sum')
data_raw_clean.head()

##### TRANSFORMACIÓN DE DATOS

In [ ]:
# Formatear el tipo de dato fecha_venta a datetime64
data_raw_clean['fecha_venta'] = pd.to_datetime(data_raw_clean['fecha_venta'], format='%Y-%m-%d')
data_flat_clean['fecha_venta'] = pd.to_datetime(data_flat_clean['fecha_venta'], format='%Y-%m-%d')
data_flat_clean['fecha_venta'].head()

1. AGREGACIÓN

In [ ]:
#

2. CONSTRUCCIÓN DE ATRIBUTOS

In [ ]:
# Creación de un nuevo atributo par tipo_venta
data_raw_clean['tipo_venta_label'] = data_raw_clean['tipo_venta'].map({0: 'Estandar', 1: 'Promocion'})

data_flat_clean['tipo_venta_label'] = data_flat_clean['tipo_venta'].map({0: 'Estandar', 1: 'Promocion'})
data_flat_clean.head()

In [ ]:
meses = {
    1: 'enero', 2: 'febrero', 3: 'marzo', 4: 'abril',
    5: 'mayo', 6: 'junio', 7: 'julio', 8: 'agosto',
    9: 'septiembre', 10: 'octubre', 11: 'noviembre', 12: 'diciembre'
}

# Extraer año, mes y día de la columna 'fecha_venta'
data_raw_clean['año'] = data_raw_clean['fecha_venta'].dt.year
data_raw_clean['mes'] = data_raw_clean['fecha_venta'].dt.month.map(meses)
data_raw_clean['día'] = data_raw_clean['fecha_venta'].dt.day
data_raw_clean['trimestre'] = data_raw_clean['fecha_venta'].dt.quarter

# Extraer año, mes y día de la columna 'fecha_venta'
data_flat_clean['año'] = data_flat_clean['fecha_venta'].dt.year
data_flat_clean['mes'] = data_flat_clean['fecha_venta'].dt.month.map(meses)
data_flat_clean['día'] = data_flat_clean['fecha_venta'].dt.day
data_flat_clean['trimestre'] = data_flat_clean['fecha_venta'].dt.quarter

# Verificar las primeras filas para asegurarnos de que se crearon correctamente
data_flat_clean[['fecha_venta', 'año', 'mes', 'día', 'trimestre']].head(100)

### TEMA 10

In [ ]:
# Productos más vendidos
producto_mas_vendidos = data_flat_clean.groupby('nombre').agg(
    total_unidades=('cantidad', 'sum'),
    total_ingresos=('total', 'sum'),
    prom_unidades=('cantidad', 'mean'),
).reset_index()

producto_mas_vendidos = producto_mas_vendidos.sort_values(by='total_unidades', ascending=False)
producto_mas_vendidos.head()

In [ ]:
# Clientes con más compras
ventas_por_cliente = data_raw_clean.groupby('cliente').agg(
    total_compras=('fecha_venta', 'count'),
    total_gasto=('pago_total', 'sum'),
    gasto_promedio=('pago_total', 'mean'),
    compras_promocion=('tipo_venta', 'mean')
).reset_index()

ventas_por_cliente = ventas_por_cliente.sort_values(by='total_compras', ascending=False)
ventas_por_cliente.head()

In [ ]:
# Agregaciones por tipo de venta
impacto_promociones = data_raw_clean.groupby('tipo_venta').agg(
    total_ventas=('tipo_venta', 'count'),
    total_ingresos=('pago_total', 'sum'),
    promedio_ingresos=('pago_total', 'mean'),
    promedio_productos=('productos', lambda x: sum(len(p) for p in x) / len(x))
).reset_index()

# Etiquetas para tipo de venta
impacto_promociones['tipo_venta'] = impacto_promociones['tipo_venta'].replace({0: 'Estándar', 1: 'Promoción'})

impacto_promociones.head()

In [ ]:
# Agregaciones por año y mes
data_raw_clean['año'] = data_raw_clean['fecha_venta'].dt.year
data_raw_clean['mes'] = data_raw_clean['fecha_venta'].dt.month

ventas_por_periodo = data_raw_clean.groupby(['año', 'mes']).agg(
    total_ingresos=('pago_total', 'sum'),
    total_ventas=('fecha_venta', 'count'),
    promedio_ingreso=('pago_total', 'mean')
).reset_index()

ventas_por_periodo.head()


In [ ]:
# Productos más vendidos por tipo de venta
productos_por_tipo = data_flat_clean.groupby(['nombre', 'tipo_venta']).agg(
    total_unidades=('cantidad', 'sum'),
    total_ingresos=('total', 'sum')
).reset_index()

productos_por_tipo['tipo_venta'] = productos_por_tipo['tipo_venta'].replace({0: 'Estándar', 1: 'Promoción'})

productos_por_tipo.head(100)


In [ ]:
# Extraer el año de la columna 'fecha_venta'
data_raw_clean['año'] = data_raw_clean['fecha_venta'].dt.year

# Agrupar por año y calcular el total de ventas
ventas_por_ano = data_raw_clean.groupby('año').agg(
    total_ventas=('pago_total', 'sum'),
).reset_index()

# Ordenar por año (opcional)
ventas_por_ano = ventas_por_ano.sort_values(by='año', ascending=True)

# Mostrar la tabla
ventas_por_ano.head()

In [ ]:
# Agrupar por año y trimestre, y calcular el total de ventas
ventas_por_trimestre = data_raw_clean.groupby(['año', 'trimestre']).agg(
    total_ventas=('pago_total', 'sum'),
).reset_index()

# Ordenar por año y trimestre (opcional)
ventas_por_trimestre = ventas_por_trimestre.sort_values(by=['año', 'trimestre'])

# Mostrar la tabla
ventas_por_trimestre.head(100)

In [ ]:
data_flat_clean.head()

### TEMA 11

#### ORDENAMOS LOS ATRIBUTOS DE LAS VARIABLES

1. DATA_RAW

In [ ]:
# Define el orden de las columnas como una lista
columnas_ordenadas = [
    'tipo_venta',  'tipo_venta_label', 'fecha_venta', 'año', 'mes', 'día', 'trimestre','productos', 'total_productos', 'pago_total', 'cliente'
]

# Reordenar las columnas en el DataFrame
data_raw_clean = data_raw_clean[columnas_ordenadas]

# Verificar el nuevo orden
print(data_raw_clean.head())

2. DATA_FLAT

In [ ]:
# Definir el orden de las columnas para data_flat_clean
columnas_ordenadas_flat = [
    'nombre', 'precio_unitario', 'cantidad', 'total', 'tipo_venta', 'tipo_venta_label', 'fecha_venta', 'año', 'mes', 'día', 'trimestre', 'pago_total', 'cliente'
]

# Reordenar las columnas en data_flat_clean
data_flat_clean = data_flat_clean[columnas_ordenadas_flat]

# Verificar el nuevo orden
print(data_flat_clean.head())

#### EXPORTACIÓN DE LOS DATOS

1. DATA_RAW

In [ ]:
# Cambiar el formato de fecha_venta a string para que se guarde de manera visible
data_raw_copy = data_raw_clean.copy()
data_raw_copy['fecha_venta'] = data_raw_copy['fecha_venta'].dt.strftime('%Y-%m-%d')

In [ ]:
# Guardar en formato JSON con indentación
data_raw_copy.to_json('data_pura.json', orient='records', lines=False)

with open('data_pura.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

with open('data_pura.json', 'w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

print("Archivo data_pura guardado exitosamente.")

2. DATA_FLAT

In [ ]:
# Cambiar el formato de fecha_venta a string para que se guarde de manera visible
data_flat_copy = data_flat_clean.copy()
data_flat_copy['fecha_venta'] = data_flat_copy['fecha_venta'].dt.strftime('%Y-%m-%d')

In [ ]:
# Guardar en formato JSON con indentación
data_flat_copy.to_json('data_desanidada.json', orient='records', lines=False)

with open('data_desanidada.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

with open('data_desanidada.json', 'w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

print("Archivo data_desanidada guardado exitosamente.")

-------